In [1]:
import pandas as pd 
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score, precision_score 
import fastapi
from typing import Optional

In [2]:


_x = [1]
_y = [1]
_prediction = ["True"]
_result = ["True"] 
model = DummyClassifier(strategy="constant", constant="True")
model.fit(np.array([_x,_y]).reshape(1, -1), _result)


DummyClassifier(constant='True', strategy='constant')

In [3]:
def model_evaluation(y_test, y_pred) :
    print('Trained Model Test Data Accuracy Score :',accuracy_score(y_test, y_pred)*100)

    y_test = [x == 'True' if isinstance(x, str) else x for x in y_test]
    y_pred = [x == 'True' if isinstance(x, str) else x for x in y_pred]
    
    testacc=accuracy_score(y_test, y_pred)
    testrecall=recall_score(y_test, y_pred)
    testprecision=precision_score(y_test, y_pred)
    print(' ')
    print(classification_report(y_test, y_pred))
    return testacc, testrecall, testprecision

In [4]:
def create_eval_df(x, y, result) :
    data = {"x": x, "y": y, "result": result}
    df = pd.DataFrame(data)
    eval_df = df.copy()
    eval_df["predict"] = model.predict(data[x,y])
    eval_df["correct"] = eval_df["predict"] == eval_df["result"]
    eval_df["correctandtrue"] = eval_df["predict"].isin([True]) & eval_df["result"].isin([True])
    return eval_df

In [5]:
def retrain_model(last_n: int, model_type: str = "Dummy", strategy: Optional[str] = None, k: Optional[int] = 3):
    global model

    if len(_x) < last_n or len(_result) < last_n:
        return {"error": "Not enough data to retrain the model."}

    X = np.array(list(zip(_x[-last_n:], _y[-last_n:])))
    Y = np.array(_result[-last_n:])

    if model_type == "Dummy":
        model = DummyClassifier(strategy=strategy or "stratified")
    elif model_type == "KNN":
        model = KNeighborsClassifier(n_neighbors=k)
    else:
        return {"error": "Unsupported model type."}

    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, stratify=Y)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    testacc, testrecall, testprecision = model_evaluation(y_test, y_pred)

    return {
        "message": f"Model retrained using {model_type}",
        "accuracy": testacc,
        "recall": testrecall,
        "precision": testprecision
    }

In [6]:
app = fastapi.FastAPI()


@app.get("/predict/")
def predict(id:int = 0 , x:float =0, y:float=0):
    print(id, x, y)
    _x.insert(id, x)
    _y.insert(id, y)
    prediction = model.predict(np.array([x,y]).reshape(1, -1))
    _prediction.insert(id, prediction[0])
    print(prediction[0])
    return dict([("prediction", prediction[0])])

@app.get("/result/")
def result(id:int, result:str):
    print(result)
    _result.insert(id, result)
    return dict([("message", "Copied result for {}".format(id))])

@app.get("/retrain/")
def retrain(last_n: int, model_type: str = "Dummy", strategy: Optional[str] = "stratified", k: Optional[int] = 3):
    return retrain_model(last_n=last_n, model_type=model_type, strategy=strategy, k=k)

@app.get("/new_model/")
def new_model(model:str, last_n:int, strategy:Optional[str] = None, k:Optional[int] = 0):
    print(model, last_n, strategy, k)
    if model == "Dummy" :
        model = DummyClassifier(strategy=strategy)
    elif model == "KNN" :
        model = KNeighborsClassifier(k=k)
    #model.set_params(model_params)

    message = retrain(last_n=last_n)

    returns = {"message": "Model trained {}".format(model), "train": message}
    print(returns)

    return returns

@app.get("/items/")
async def read_item(skip: int = 0, limit: int = 10):
    return {"skip": skip, "limit": limit}

#@app.get("/evaluate_model/")
#def evaluate_model(last_n:int):
#    return {"message": "Item created", "last_n": last_n}

In [7]:
#print(predict(1, 1, 1))
#result(1, True)
new_model("Dummy", 50, "stratified")

Dummy 50 stratified 0
{'message': "Model trained DummyClassifier(strategy='stratified')", 'train': {'error': 'Not enough data to retrain the model.'}}


{'message': "Model trained DummyClassifier(strategy='stratified')",
 'train': {'error': 'Not enough data to retrain the model.'}}

In [8]:
@app.get("/items/")
async def read_item(skip: int = 0, limit: int = 10):
    return {"skip": skip, "limit": limit}

In [ ]:
import asyncio
import uvicorn

if __name__ == "__main__":
    config = uvicorn.Config(app)
    server = uvicorn.Server(config)
    loop = asyncio.get_running_loop()
    loop.create_task(server.serve())

INFO:     Started server process [9624]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


1 -0.9106631645779717 -0.10464963706820685
True
INFO:     127.0.0.1:65121 - "GET /predict/?id=1&x=-0.9106631645779717&y=-0.10464963706820685 HTTP/1.1" 200 OK
True
INFO:     127.0.0.1:65124 - "GET /result/?id=1&result=True HTTP/1.1" 200 OK
INFO:     127.0.0.1:65127 - "GET /retrain/?last_n=10&model_type=KNN&strategy=stratified&k=5 HTTP/1.1" 200 OK
2 0.915535882863248 0.640947014985138
True
INFO:     127.0.0.1:65220 - "GET /predict/?id=2&x=0.915535882863248&y=0.640947014985138 HTTP/1.1" 200 OK
True
INFO:     127.0.0.1:65223 - "GET /result/?id=2&result=True HTTP/1.1" 200 OK
3 -0.0007331859337078583 0.6830385888460697
True
INFO:     127.0.0.1:65226 - "GET /predict/?id=3&x=-0.0007331859337078583&y=0.6830385888460697 HTTP/1.1" 200 OK
True
INFO:     127.0.0.1:65229 - "GET /result/?id=3&result=True HTTP/1.1" 200 OK
4 -0.7547301339556327 -0.064858490896196
True
INFO:     127.0.0.1:65232 - "GET /predict/?id=4&x=-0.7547301339556327&y=-0.064858490896196 HTTP/1.1" 200 OK
True
INFO:     127.0.0.1:652

c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMet

12 0.5868171649661642 0.46404787810778836
False
INFO:     127.0.0.1:65290 - "GET /predict/?id=12&x=0.5868171649661642&y=0.46404787810778836 HTTP/1.1" 200 OK
True
INFO:     127.0.0.1:65293 - "GET /result/?id=12&result=True HTTP/1.1" 200 OK
13 0.4902878725465374 -0.660232986267391
False
INFO:     127.0.0.1:65296 - "GET /predict/?id=13&x=0.4902878725465374&y=-0.660232986267391 HTTP/1.1" 200 OK
False
INFO:     127.0.0.1:65299 - "GET /result/?id=13&result=False HTTP/1.1" 200 OK
14 0.5388501477923184 0.4299096871855439
False
INFO:     127.0.0.1:65302 - "GET /predict/?id=14&x=0.5388501477923184&y=0.4299096871855439 HTTP/1.1" 200 OK
True
INFO:     127.0.0.1:65305 - "GET /result/?id=14&result=True HTTP/1.1" 200 OK
15 -0.2541930628048723 -0.3609445008447354
False
INFO:     127.0.0.1:65308 - "GET /predict/?id=15&x=-0.2541930628048723&y=-0.3609445008447354 HTTP/1.1" 200 OK
True
INFO:     127.0.0.1:65311 - "GET /result/?id=15&result=True HTTP/1.1" 200 OK
16 -0.16035273857402887 0.7719672643934796
F

c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

False
INFO:     127.0.0.1:49971 - "GET /result/?id=31&result=False HTTP/1.1" 200 OK
Trained Model Test Data Accuracy Score : 50.0
 


c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

       False       0.00      0.00      0.00         2
        True       0.50      1.00      0.67         2

    accuracy                           0.50         4
   macro avg       0.25      0.50      0.33         4
weighted avg       0.25      0.50      0.33         4

INFO:     127.0.0.1:49974 - "GET /retrain/?last_n=10&model_type=KNN&strategy=stratified&k=5 HTTP/1.1" 200 OK


c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Trained Model Test Data Accuracy Score : 50.0
 
              precision    recall  f1-score   support

       False       0.50      0.25      0.33         4
        True       0.50      0.75      0.60         4

    accuracy                           0.50         8
   macro avg       0.50      0.50      0.47         8
weighted avg       0.50      0.50      0.47         8

INFO:     127.0.0.1:50000 - "GET /retrain/?last_n=20&model_type=Dummy&strategy=stratified&k=3 HTTP/1.1" 200 OK
